In [ ]:
import csv,datetime
import pandas as pd

import math

#DSEA 2019-2020 Project
#Objective: Scrape csv training data for meaningful insights
#Write CSV containing trainable data
#Scrape 1:
#Col 1: Student ID
#Col 2: Time spent on part A
#Col 3: # of questions completed
#Col 4: # of actions
#Col 5: Target Variable
############################################
#Scrape 2:
#Col 1: Student ID
#Col 2: MCSS Average Time
#Col 3: GridMS Average Time
#Col 4: MatchMS Average Time
#Col 5: ZonesMS Average Time
#Col 6: FillInBlank Average Time
#Col 7: CompositeCR Average Time
#Col 8: Target Variable
#############################################
#Scrape 3:
#Col 1: Student ID
#Col 2: Average time spent per question
#Col 3: Number of questions opened
#Col 4: Count of specific actions taken
#Col 5-Col 10: Time spent on each question
#############################################
#Scrape 5:
#Click Rate at 10 click intervals - check
#Time count for observables - check
#Time of Ending - CHECK
#Calculation Time open and close average
#Average Time Reviewing
#Above or below quantile per question

#Scrape 6:
#Action Rates per minute

#Scrape 7:
#Clicks per question
#Clicks per item type

def scrape1(trainData, labelData, outCSV):
    curStudent = labelData.iloc[0,:]
    firstStudent = trainData.iloc[0,:]
    startTime = parseTime(firstStudent['EventTime'])
    endTime = parseTime(firstStudent['EventTime'])
    nonQuestions = ['Directions','BlockReview','TimeLeftMessage','Help']
    actionCount = 0
    questionList = []
    entryList = []
    studentCount = 0
    rowCount= 0

    for row in trainData.iterrows():
        rowCount +=1
        if(rowCount % 10000 == 0):
            print (rowCount)
        actionCount +=1

        #break between students - Write to row
        if((row[1].loc['STUDENTID'] != curStudent.loc['STUDENTID'])) or row[0] == len(trainData)-1:
            entryList.append({'StudentID':curStudent.loc['STUDENTID'],'TotTime':(endTime-startTime).total_seconds(),'NumQuests':len(questionList),'NumActs':actionCount,'TargetV':curStudent['Target']})

            studentCount+=1
            #write current data line
            print(curStudent.loc['STUDENTID'], (endTime-startTime).total_seconds(), len(questionList), actionCount, curStudent['Target'])

            #update tracking vars for next student if not last student
            if(row[0] != len(trainData)-1): curStudent = labelData.iloc[studentCount,:]
            startTime = parseTime(row[1].loc['EventTime'])
            endTime = startTime
            questionList = []
            actionCount=0
            questionCount=0

        #break between questions
        elif(row[1].loc['AccessionNumber'] != trainData.iloc[row[0]-1,:].loc['AccessionNumber']):
            if(not row[1].loc['AccessionNumber'] in questionList and not row[1].loc['ItemType'] in nonQuestions):
                questionList.append(row[1].loc['AccessionNumber'])

        if(str(row[1].loc['EventTime']) != 'nan'): endTime=parseTime(row[1].loc['EventTime'])






    outData = pd.DataFrame(entryList, columns =['StudentID','TotTime','NumQuests','NumActs','TargetV'])
    outData.to_csv(outCSV,index=False)

def scrape3(trainData, outCSV, labelData=None):
    #initialize all tracking variables
    curStudent = trainData.iloc[0,:]
    #labelStudent = labelData.iloc[0,:]
    startTime=endTime=qStart=qEnd = parseTime(curStudent['EventTime'])
    #nonQuestions = ['Directions','BlockReview','TimeLeftMessage','Help']
    actionCount = 0
    curData = {'StudentID' : curStudent['STUDENTID'], 'TargetV':True}
    entryList = []
    revisits = 0

    questionCount = 0
    studentCount = 0
    rowCount= 0

    for row in trainData.iterrows():
        rowCount +=1
        if(rowCount % 10000 == 0):
            print(rowCount)
        actionCount +=1


        #print(rowCount)
        #break between students - Write to row
        if((row[1].loc['STUDENTID'] != curStudent.loc['STUDENTID'])) or row[0] == len(trainData)-1:
            curData['NumActs'] = actionCount
            curData['NumQuests'] = questionCount
            curData['Revisits'] = revisits
            entryList.append(curData)
            studentCount+=1

            #update tracking vars for next student if not last student
            if(row[0] != len(trainData)-1): curStudent = row[1]
            startTime=endTime=qStart=qEnd = parseTime(row[1].loc['EventTime'])

            curData = {'StudentID' : curStudent['STUDENTID'], 'TargetV':True}
            actionCount=0
            revisits = 0
            questionCount=0

        #break between questions
        elif(row[1].loc['AccessionNumber'] != trainData.iloc[row[0]-1,:].loc['AccessionNumber']):

            #if we have NOT seen this question before
            if(row[1].loc['AccessionNumber'] not in curData.keys()):
                questionCount +=1
                curData[row[1].loc['AccessionNumber']] = (qEnd-qStart).total_seconds()
            else:
                revisits +=1
                curData[row[1].loc['AccessionNumber']] += (qEnd-qStart).total_seconds()

            if(row[1].loc['ItemType'] not in curData.keys()):
                curData[row[1].loc['ItemType']] = (qEnd - qStart).total_seconds()
            else:
                curData[row[1].loc['ItemType']] += (qEnd-qStart).total_seconds()

            #update the starting time for a new question
            if(str(row[1].loc['EventTime']) != 'nan'): qStart=qEnd=parseTime(row[1].loc['EventTime'])

         #update the ending time with each row
        if(str(row[1].loc['EventTime']) != 'nan'): endTime=qEnd=parseTime(row[1].loc['EventTime'])

        #Add new observable
        if(row[1].loc['Observable'] not in curData.keys()): curData[row[1].loc['Observable']] =1

        #Add to the count of this observable
        else: curData[row[1].loc['Observable']] +=1

    outData = pd.DataFrame(entryList)
    outData.to_csv(outCSV,index=False)

def scrape4(trainData, procData, labelData, outCSV):
    labelDict = {}
    dataList = []
    for row in trainData.itertuples():
        #break between students
        dataList.append([row.STUDENTID,row.ItemType,row.Observable])
    for row in labelData.itertuples():
        labelDict[row.STUDENTID] = row.Target

    curID = dataList[0][0]
    count = 0
    curActions = []
    studentList =[]
    finalList = []
    for i in dataList:
        #add all actions for current student to curActions
        if(curID == i[0]):
            curActions.append(i)
        #break between students
        else:
            studentList.append([curID,curActions])
            curActions = []
            curID = i[0]
    studentList.append([curID,curActions])
    procList = []
    for row in procData.itertuples():
        procList.append(row.Process)

    i = j = 0
    while(i<len(studentList)):
        commonProcs = {}
        print(i)
        j=0
        #print(len(studentList))
        #print(i)
        while(j<len(procList)):
            commonProcs = compareProcs2(commonProcs,studentList[i][1],procList[j])
            j+=1
        commonProcs['StudentID'] = studentList[i][0]
        commonProcs['TargetV'] = labelDict[studentList[i][0]]
        finalList.append(commonProcs)
        i+=1
    outData = pd.DataFrame(finalList)
    outData.to_csv(outCSV,index=False)

def scrape5(trainData, outCSV, labelData=None):
    #initialize all tracking variables
    curStudent = trainData.iloc[0,:]
    #labelStudent = labelData.iloc[0,:]
    actionTimerEnd=actionTimer=startTime=endTime=qStart=qEnd = parseTime(curStudent['EventTime'])
    #nonQuestions = ['Directions','BlockReview','TimeLeftMessage','Help']
    actionCount = 0
    obsTimer = 0
    reviewTime = 0
    curData = {'StudentID' : curStudent['STUDENTID'], 'TargetV':True, 'StartTime':parseTime(curStudent['EventTime']).total_seconds()}
    entryList = []

    questionCount = 0
    studentCount = 0
    rowCount= 0
    priorObs = startTime

    for row in trainData.iterrows():
        rowCount +=1
        if(rowCount % 10000 == 0):
            print(rowCount)
        actionCount +=1

        observed = row[1].loc['Observable']


        #print(rowCount)
        #break between students - Write to row
        if((row[1].loc['STUDENTID'] != curStudent.loc['STUDENTID'])) or row[0] == len(trainData)-1:
            entryList.append(curData)
            studentCount+=1

            #update tracking vars for next student if not last student
            if(row[0] != len(trainData)-1): curStudent = row[1]
            actionTimer=startTime=endTime=qStart=qEnd = parseTime(row[1].loc['EventTime'])

            curData = {'StudentID' : curStudent['STUDENTID'], 'TargetV':True}
            curData['StartTime'] = parseTime(row[1].loc['EventTime']).total_seconds()
            actionCount=0
            priorObs = startTime
            questionCount=0
            reviewTime = 0

        #break between questions
        elif(row[1].loc['AccessionNumber'] != trainData.iloc[row[0]-1,:].loc['AccessionNumber']):

            #update the starting time for a new question
            if(str(row[1].loc['EventTime']) != 'nan'): qStart=qEnd=parseTime(row[1].loc['EventTime'])

         #update the ending time with each row
        if(str(row[1].loc['EventTime']) != 'nan'): endTime=qEnd=actionTimerEnd=obsTimer=parseTime(row[1].loc['EventTime'])

        #Add new observable
        if(row[1].loc['Observable'] not in curData.keys()):
            curData[row[1].loc['Observable']] =1
            curData[row[1].loc['Observable']+"_T"] = (obsTimer-priorObs).total_seconds()

        #Add to the count of this observable
        else:
            curData[row[1].loc['Observable']] +=1
            curData[row[1].loc['Observable']+"_T"] += (obsTimer-priorObs).total_seconds()


        #add time for last 10 actions
        if(actionCount%10 == 0 and actionCount<600):
            curData["actionRate" +str(actionCount)] = (actionTimerEnd-actionTimer).total_seconds()
            actionTimer=actionTimerEnd=parseTime(row[1].loc['EventTime'])

        priorObs =obsTimer

    outData = pd.DataFrame(entryList)
    outData.to_csv(outCSV,index=False)
 
def scrape6(trainData, outCSV, labelData=None):
    #initialize all tracking variables
    curStudent = trainData.iloc[0,:]
    #labelStudent = labelData.iloc[0,:]
    actionTimerEnd=actionTimer=startTime=endTime=qStart=qEnd = parseTime(curStudent['EventTime'])
    #nonQuestions = ['Directions','BlockReview','TimeLeftMessage','Help']
    actionCount = 0
    actionMark = 0
    obsTimer = 0
    reviewTime = 0
    curData = {'StudentID' : curStudent['STUDENTID']}
    entryList = []

    questionCount = 0
    studentCount = 0
    rowCount= 0
    priorObs = startTime

    for row in trainData.iterrows():
        rowCount +=1
        if(rowCount % 10000 == 0):
            print(rowCount)
        actionCount +=1

        observed = row[1].loc['Observable']


        #print(rowCount)
        #break between students - Write to row
        if((row[1].loc['STUDENTID'] != curStudent.loc['STUDENTID'])) or row[0] == len(trainData)-1:
            entryList.append(curData)
            studentCount+=1

            #update tracking vars for next student if not last student
            if(row[0] != len(trainData)-1): curStudent = row[1]
            actionTimer=startTime=actionTimerEnd = parseTime(row[1].loc['EventTime'])

            curData = {'StudentID' : curStudent['STUDENTID']}
            actionCount=0
            actionMark = 0


         #update the ending time with each row
        if(str(row[1].loc['EventTime']) != 'nan'): endTime=qEnd=actionTimerEnd=obsTimer=parseTime(row[1].loc['EventTime'])

        #add time for last 10 actions
        if((actionTimerEnd - actionTimer).total_seconds()>60):
            curData["clickRate" +str(actionMark)] = actionCount
            actionTimer=actionTimerEnd=parseTime(row[1].loc['EventTime'])
            actionMark +=1
            actionCount = 0

        priorObs =obsTimer

    outData = pd.DataFrame(entryList)
    outData.to_csv(outCSV,index=False)

def scrape7(trainData, outCSV, labelData=None):
    #initialize all tracking variables
    curStudent = trainData.iloc[0,:]
    actionCount = 0

    obsTimer = 0
    reviewTime = 0
    curData = {'StudentID' : curStudent['STUDENTID']}
    entryList = []

    rowCount= 0

    for row in trainData.iterrows():
        rowCount +=1
        if(rowCount % 10000 == 0):
            print(rowCount)
        actionCount +=1

        observed = row[1].loc['Observable']


        #print(rowCount)
        #break between students - Write to row
        if((row[1].loc['STUDENTID'] != curStudent.loc['STUDENTID'])) or row[0] == len(trainData)-1:
            entryList.append(curData)

            #update tracking vars for next student if not last student
            if(row[0] != len(trainData)-1): curStudent = row[1]
            actionTimer=startTime=actionTimerEnd = parseTime(row[1].loc['EventTime'])

            curData = {'StudentID' : curStudent['STUDENTID']}

        # Add new ItemType
        if (row[1].loc['ItemType']+"ObsClicks" not in curData.keys()):
            curData[row[1].loc['ItemType']+"ObsClicks"] = 1

            # Add to the count of this observable
        else:
            curData[row[1].loc['ItemType']+"ObsClicks"] += 1

        # Add new Question
        if (row[1].loc['AccessionNumber']+"QClicks" not in curData.keys()):
            curData[row[1].loc['AccessionNumber']+"QClicks"] = 1

            # Add to the count of this observable
        else:
            curData[row[1].loc['AccessionNumber']+"QClicks"] += 1


    outData = pd.DataFrame(entryList)
    outData.to_csv(outCSV,index=False)



#convert time stamp into timeDelta object for arithmetic operations
def parseTime(eventTime):
    timeString = eventTime.split()[1].split(':')
    t = datetime.timedelta(hours=int(timeString[0]), minutes=int(timeString[1]), seconds=int(timeString[2].split('.')[0]), milliseconds=int(timeString[2].split('.')[1]))
    return t

def timeCut(trainData, cutTime, cutCSV, outData=None):
    outData = pd.DataFrame(columns=['STUDENTID','Block','AccessionNumber','ItemType','Observable','ExtendedInfo','EventTime'])
    dataList = []
    curStudent = trainData.iloc[0,:]
    firstStudent = trainData.iloc[0,:]
    startTime = parseTime(firstStudent['EventTime'])
    endTime = parseTime(firstStudent['EventTime'])
    rowNum = 0
    for row in trainData.iterrows():
        rowNum += 1
        print(rowNum)

        if ((row[1].loc['STUDENTID'] != curStudent.loc['STUDENTID'])):
            curStudent = row[1]
            startTime = parseTime(row[1].loc['EventTime'])
            endTime = startTime
        if (str(row[1].loc['EventTime']) != 'nan'):
            endTime = parseTime(row[1].loc['EventTime'])
            if((endTime-startTime).total_seconds()<cutTime):
                dataList.append(row[1])

    outData = pd.DataFrame(dataList,columns=['STUDENTID','Block','AccessionNumber','ItemType','Observable','ExtendedInfo','EventTime'])
    outData.to_csv(cutCSV,index=False)




def processListScrape(trainData, procList, outCSV):
    dataList = []
    for row in trainData.itertuples():
        #break between students
        dataList.append([row.STUDENTID, row.Observable,row.EventTime])

    curID = dataList[0][0]
    count = 0
    curActions = []
    studentList =[]
    finalList = []
    for i in dataList:
        #add all actions for current student to curActions
        if(curID == i[0]):
            curActions.append(i)
        #break between students
        else:
            studentList.append([curID,curActions])
            curActions = []
            curID = i[0]
    studentList.append([curID,curActions])

    i = j = 0
    while(i<len(studentList)):
        commonProcs = {}
        #print(i)
        j=0
        #print(len(studentList))
        if(i%100 == 0): print(i)
        while(j<len(procList)):
            commonProcs = compareProcs2(commonProcs,studentList[i][1],procList[j])
            j+=1
        commonProcs['StudentID'] = studentList[i][0]
        #commonProcs['TargetV'] = labelDict[studentList[i][0]]
        finalList.append(commonProcs)
        i+=1
    outData = pd.DataFrame(finalList)
    outData.to_csv(outCSV,index=False)

def procMine(trainData, outCSV):
    studentCount = 0
    dataList = []
    rowCount = 0
    for row in trainData.itertuples():
        rowCount+=1
        #break between students
        dataList.append([row.StudentID,row.ItemType,row.Observable])
        #print(rowCount)


    curID = dataList[0][0]
    count = 0
    curActions = []
    studentList =[]
    for i in dataList:
        #add all actions for current student to curActions
        if(curID == i[0]):
            curActions.append(i)
        #break between students
        else:
            studentList.append(curActions)
            curActions = []
            curID = i[0]


    i = j = 0
    commonProcs = {}
    while(i<100):
        j=i+1
        print(len(studentList))
        print(i)
        while(j<100):
            commonProcs = compareStudents(commonProcs,studentList[i],studentList[j],10)
            j+=1
        i+=1

    df = pd.DataFrame.from_dict(commonProcs, orient='index')
    print(df)
    df.to_csv(outCSV,index=True)
def procDiff(trueData, falseData, outCSV):
    studentCount = 0
    trueList = []
    rowCount = 0
    falseList = []
    finalDict = {}
    for row in trueData.itertuples():
        #break between students
        trueList.append([row.Process,row.Count])
        #print(rowCount)

    for row in falseData.itertuples():
        falseList.append([row.Process,row.Count])
    i=j=0
    while(i<len(trueList)):
        j=0
        if(i%100 ==0): print(i)
        while(j<len(falseList)):
            if(trueList[i][0] == falseList[j][0]):
                finalDict[trueList[i][0]] = [trueList[i][1], falseList[j][1], trueList[i][1] - falseList[j][1]]
            j+=1

        i+=1
    for i in trueList:
        if(not i[0] in finalDict.keys()):
            finalDict[i[0]] = [i[1],0,i[1]]

    for i in falseList:
        if(not i[0] in finalDict.keys()):
            finalDict[i[0]] = [0,i[1],0-i[1]]

    df = pd.DataFrame.from_dict(finalDict, orient='index')
    print(df)
    df.to_csv(outCSV,index=True)



def procMine2(trainData,procData, outCSV):
    studentCount = 0
    dataList = []
    rowCount = 0
    procList = []
    for row in trainData.itertuples():
        rowCount+=1
        #break between students
        dataList.append([row.StudentID,row.ItemType,row.Observable])
        #print(rowCount)

    for row in procData.itertuples():
        procList.append(row.Process)



    curID = dataList[0][0]
    count = 0
    curActions = []
    studentList =[]
    for i in dataList:
        #add all actions for current student to curActions
        if(curID == i[0]):
            curActions.append(i)
        #break between students
        else:
            studentList.append(curActions)
            curActions = []
            curID = i[0]


    i = j = 0
    commonProcs = {}
    while(i<len(procList)):
        print(i)
        j=0
        #print(len(studentList))
        #print(i)
        while(j<len(studentList)):
            commonProcs = compareProcs(commonProcs,studentList[j],procList[i])
            j+=1
        i+=1

    df = pd.DataFrame.from_dict(commonProcs, orient='index')
    print(df)
    df.to_csv(outCSV,index=True)

def compareStudents(commonProcs, stuA, stuB, limit):
    list1 = []
    list2 = []
    actLen = 5
    while(actLen < limit):
        i=0
        j=0
        while(i + actLen < len(stuA)):

            list1 = stuA[i:i + actLen]
            #print(list1)
            str1 = stringify(list1)
            j=0
            while(j+actLen<len(stuB)):
                list2 = stuB[j:j+actLen]
                str2 = stringify(list2)
                if(str1 == str2):
                    if(str1 in commonProcs.keys()):
                        commonProcs[str1] += 1
                    else:
                        commonProcs[str1] = 1
                j+= 1
            i+=1
        actLen +=1
    return commonProcs

def compareProcs(commonProcs,stu,proc):
    list1 = []
    list2 = []
    procLen = proc.count(';')
    #print(proc)
    #print(procLen)
    i = 0
    while(i + procLen < len(stu)):
        list1 = stu[i:i + procLen]
        #print(list1)
        str1 = stringify(list1)
        #print(str1)
        if(not proc in commonProcs.keys()):
            commonProcs[proc] = 0
        if(str1 == proc):
            if(str1 in commonProcs.keys()):
                commonProcs[str1] += 1
            else:
                commonProcs[str1] = 1
        i+=1
    return commonProcs

#compare list of observables only
def compareProcs2(commonProcs,stu,proc):
    list1 = []
    list2 = []
    procLen = proc.count(';') + 1
    #print(proc)
    #print(procLen)
    i = 0
    if (not proc in commonProcs.keys()):
        commonProcs[proc] = 0
        commonProcs[proc + "_PT"] = 0
    while(i + procLen < len(stu)):
        list1 = stu[i:i + procLen]
        #print(list1)
        str1 = stringify2(list1)
        time = timeify(list1)
        if(str1 == proc):
            #print("Process:", proc, "String:", str1)
            commonProcs[str1] += 1
            commonProcs[str1 + "_PT"] += time
        i+=1
    if(commonProcs[proc] != 0): commonProcs[proc+'_PT_R'] = commonProcs[proc+"_PT"]/commonProcs[proc]
    return commonProcs

def stringify(acts):
    result = ""
    for i in acts:
        result = result + i[1] + ";" + i[2]
    return result

def stringify2(acts):
    result = ""
    for i in acts:
        result = result + i[1] + ";"
    return result[:-1]

def timeify(acts):
    result = 0
    startTime = parseTime(acts[0][2])
    endTime = parseTime(acts[len(acts)-1][2])
    return (endTime-startTime).total_seconds()





def tFSplit(trainData, labelData, trueCSV, falseCSV):
    curStudent = labelData.iloc[0, :]
    studentCount = 0
    falseList = []
    trueList = []
    rowCount = 0

    for row in trainData.iterrows():
        rowCount += 1
        if (rowCount % 30000 == 0):
            print(rowCount)
        # print(rowCount)
        if((row[1].loc['STUDENTID'] != curStudent.loc['STUDENTID'])) or row[0] == len(trainData)-1:
            studentCount += 1
            if (row[0] != len(trainData) - 1): curStudent = labelData.iloc[studentCount, :]

        curDict = row[1].to_dict()
        if(curStudent.loc['Target'] == True):
            curDict['TargetV'] = True
            trueList.append(curDict)
        else:
            curDict['TargetV'] = False
            falseList.append(curDict)

    trueData = pd.DataFrame(trueList)
    falseData = pd.DataFrame(falseList)
    trueData.to_csv(trueCSV, index=False)
    falseData.to_csv(falseCSV, index=False)



def hiddenScrape(trainData, outCSV):
    curStudent = trainData.iloc[0,:]
    labelStudent = trainData.iloc[0,:]
    startTime=endTime=qStart=qEnd = parseTime(curStudent['EventTime'])
    nonQuestions = ['Directions','BlockReview','TimeLeftMessage','Help']
    actionCount = 0
    curData = {'StudentID' : labelStudent['STUDENTID']}
    entryList = []
    questionCount = 0
    studentCount = 0
    rowCount= 0

    for row in trainData.iterrows():
        rowCount +=1
        if(rowCount % 10000 == 0):
            print(rowCount)
        actionCount +=1


        #print(rowCount)
        #break between students - Write to row
        if((row[1].loc['STUDENTID'] != curStudent.loc['STUDENTID'])) or row[0] == len(trainData)-1:
            curData['NumActs'] = actionCount
            curData['NumQuests'] = questionCount
            entryList.append(curData)

            studentCount+=1
            #write current data line
            #print(questionList)

            #update tracking vars for next student if not last student
            if(row[0] != len(trainData)-1): curStudent = row[1]
            startTime=endTime=qStart=qEnd = parseTime(row[1].loc['EventTime'])

            curData = {'StudentID' : curStudent['STUDENTID']}
            actionCount=0
            questionCount=0

        #break between questions
        elif(row[1].loc['AccessionNumber'] != trainData.iloc[row[0]-1,:].loc['AccessionNumber']):
            if(row[1].loc['AccessionNumber'] not in curData.keys()):
                questionCount +=1
                curData[row[1].loc['AccessionNumber']] = (qEnd-qStart).total_seconds()
            else:
                curData[row[1].loc['AccessionNumber']] += (qEnd-qStart).total_seconds()
            if(str(row[1].loc['EventTime']) != 'nan'): qStart=qEnd=parseTime(row[1].loc['EventTime'])
        if(str(row[1].loc['EventTime']) != 'nan'): endTime=qEnd=parseTime(row[1].loc['EventTime'])
        if(row[1].loc['Observable'] not in curData.keys()): curData[row[1].loc['Observable']] =1
        else: curData[row[1].loc['Observable']] +=1

    outData = pd.DataFrame(entryList)
    outData.to_csv(outCSV,index=False)
    
def dataProc(input, output):
    with open(input,newline='') as data:
        df = pd.read_csv(data)
        df.fillna(value=0,inplace=True)
        df.to_csv(output,index=False)




procs = ["Draw;Scratchwork Erase Mode On;Erase",
         "Scratchwork Mode Off;Vertical Item Scroll;Eliminate Choice",
         "Close Calculator;Open Calculator",
         "Lose Focus;Click Progress Navigator",
         "Receive Focus;Equation Editor Button",
         "Vertical Item Scroll;Lose Focus",
         "Vertical Item Scroll;Open Calculator;Click Choice",
         "Enter Item;Open Calculator",
         "Enter Item;Click Choice",
         "Enter Item;Receive Focus",
         "Enter Item;Eliminate Choice",
         "Enter Item;Next",
         "Enter Item;Vertical Item Scroll;Open Calculator",
         "Enter Item;Open Calculator;Calculator Buffer",
         "Enter Item;DropChoice",
         "Enter Item;Scratchwork Mode On",
         "Enter Item;Click Progress Navigator",
         "Enter Item;Increase Zoom",
         "Move Calculator;Exit Item",
         "Move Calculator;DropChoice",
         "Move Calculator;Receive Focus",
         "Move Calculator;Click Choice",
         "Open Calculator;Click Choice",
         "Close Calculator;Open Calculator",
         "Receive Focus;Lose Focus",
         "Vertical Item Scroll;Open Calculator",
         "Exit Item;Enter Item;Click Choice",
         "Exit Item;Enter Item",
         "Exit Item;Enter Item;Next",
         "Math Keypress;First Text Change",
         "Draw;Draw"]
procs2=["Enter Item;Next;Exit Item",
        "Exit Item;Calculator Buffer;Close Calculator;Click Choice",
        "Close Calculator;Click Choice",
        "Exit Item;Calculator Buffer",
        "Enter Item;Open Calculator;Move Calculator;DropChoice",
        "Enter Item;Open Calculator;Move Calculator",
        "Click Choice;Next;Exit Item",
        "Click Choice;Next",
        "Enter Item;Eliminate Choice;Click Choice",
        "Vertical Item Scroll;Next",
        "Move Calculator;Eliminate Choice",
        "Click Choice;Click Progress Navigator",
        "Click Progress Navigator;Exit Item",
        "DropChoice;DropChoice",
        "Calculator Buffer;Enter Item",
        "Move Calculator;Move Calculator",
        "Clear Answer;Click Choice",
        "Exit Item;Calculator Buffer;Close Calculator",
        "Receive Focus;Math Keypress;Lose Focus",
        "Receive Focus;Math Keypress",
        "Draw;Clear Scratchwork"]

with open('data_a_train_10.csv', newline='') as trainFile:
    trainReader = pd.read_csv(trainFile)
    processListScrape(trainReader, procs2, "ProcScrape_T10.csv")
with open('data_a_train_20.csv', newline='') as trainFile:
    trainReader = pd.read_csv(trainFile)
    processListScrape(trainReader, procs2, "ProcScrape_T20.csv")
with open('data_a_train_30.csv', newline='') as trainFile:
    trainReader = pd.read_csv(trainFile)
    processListScrape(trainReader, procs2, "ProcScrape_T30.csv")

with open('data_a_hidden_10.csv', newline='') as trainFile:
    trainReader = pd.read_csv(trainFile)
    processListScrape(trainReader, procs2, "ProcScrape_H10.csv")

with open('data_a_hidden_20.csv', newline='') as trainFile:
    trainReader = pd.read_csv(trainFile)
    processListScrape(trainReader, procs2, "ProcScrape_H20.csv")

with open('data_a_hidden_30.csv', newline='') as trainFile:
    trainReader = pd.read_csv(trainFile)
    processListScrape(trainReader, procs2, "ProcScrape_H30.csv")

